## Analyze the cube and perform the OLAP operations on that cube.

In [23]:
#libraries
import pandas as pd
import numpy as np
import random as rd
import os

In [24]:
# Setup the dimension table

#Dataset 1
dataset1 = {"key":["CIVIC", "Corolla", "Cultus","Aqua","Swift"],
         "price":[4200000, 3700000, 2100000,2500000,2800000]}
DS1 = pd.DataFrame(dataset1)


In [25]:
#Dataset2
dataset2 = {"key":range(1,13),
         "desc":["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
         "quarter":["Q1","Q1","Q1","Q2","Q2","Q2","Q3","Q3","Q3","Q4","Q4","Q4"]}
DS2 = pd.DataFrame(dataset2)

In [26]:
#Dataset3
dataset3 = {"key":["Latifabad", "Gulshan", "Iqbal Town", "Gahli", "Road","Ny","AX"],
        "name":["Hyderbad", "Karachi", "Lahore", "Deheli", "Kolkta","New Yok","California"],
        "country":["Pakistan", "Pakistan", "Pakistan", "India", "India","USA","USA"]}
DS3 = pd.DataFrame(dataset3)

In [27]:
# Function to generate the Sales table
def gen_sales(no_of_recs):
    
    # Generate transaction data randomly
    loc = rd.choices(DS3.key,weights = [2,2,3,3, 1, 1, 1],k=no_of_recs)
    
    time_month = rd.choices(DS2.key,k=no_of_recs)
    time_year = rd.choices([2020,2021,2022],k=no_of_recs)
    
    prod = rd.choices(DS1.key,k= no_of_recs,weights = [1, 3, 2,4,5])
    unit = rd.choices([1,2,3],k=no_of_recs,weights = [10, 3,40])
    
    sales = pd.DataFrame({'month':time_month,
                      'year':time_year,
                      'location':loc,
                      'Car':prod,
                      'unit':unit
                         }
                      )
    

    # Sort the records by time order
    sales = sales.sort_values(by = ['year','month'])
    
    return(sales)

In [28]:
#set columns
Purchased = gen_sales(500)
Purchased["value"] = 1
Purchased['quarter'] = 1
Purchased['country'] = 1

In [29]:
#set price
def amount(cols):
    value = cols[0]
    Car = cols[1]
    while value == 1:
        if  Car == "CIVIC":
            return 4200000
        elif Car == "Corolla":
            return 3700000
        
        elif Car == "Cultus":
            return 2100000
        
        elif Car == "Aqua":
            return 2500000 
        
        else:
            return 2800000
        
#set quaters       
def quarter(cols):
    quarter = cols[0]
    month = cols[1]
    while quarter == 1:
        
        if  month <= 3:
            return 'Q1'
        
        elif month <= 6:
            return 'Q2'
        
        elif month <= 9:
            return 'Q3'
        
        else:
            return 'Q4' 

#set countries
def country(cols):
    
    country = cols[0]
    location = cols[1]
    
    while country == 1:
        
        if  location == "Latifabad":
            return 'Pakistan'
        
        elif location == "Gulshan":
            return 'Pakistan'
        
        elif location == "Iqbal Town":
            return 'Pakistan'
        
        elif location == "Ghali":
            return 'India'
        
        elif location == "Road":
            return 'India'
        
        elif location == "Ny":
            return 'USA'
        
        elif location == "Ax":
            return 'USA'
        
        else:
            return 'USA'

In [30]:
#set quarter based on months
Purchased["quarter"] = Purchased[['quarter','month']].apply(quarter,axis=1)

#set country based on location and country name
Purchased["country"] = Purchased[['country','location']].apply(country,axis=1)

#set value of car based car models
Purchased["value"] = Purchased[['value','Car']].apply(amount,axis=1)

#set price of car
Purchased['amount'] =Purchased["value"]*Purchased['unit']

In [31]:
#convert datasets into new single dataset
Purchased.to_csv('sales.csv')


sale = os.path.abspath("sales.csv")

#read new dataset
revenue =  pd.read_csv(sale)

In [32]:
#drop unnamed 
revenue.drop(['Unnamed: 0','unit','value'],axis=1,inplace=True)

revenue = revenue[['month', 'quarter','year','location','country','Car','amount']]

In [33]:
#print some starting rows
revenue.head(20)

,month,quarter,year,location,country,Car,amount
0,1,Q1,2020,Gulshan,Pakistan,Swift,8400000
1,1,Q1,2020,Road,India,Aqua,7500000
2,1,Q1,2020,Gulshan,Pakistan,Aqua,7500000
3,1,Q1,2020,Ny,USA,Aqua,7500000
4,1,Q1,2020,AX,USA,Swift,8400000
5,1,Q1,2020,Gahli,USA,Aqua,7500000
6,1,Q1,2020,Gahli,USA,Corolla,11100000
7,1,Q1,2020,Latifabad,Pakistan,Corolla,11100000
8,1,Q1,2020,Road,India,Aqua,5000000
9,1,Q1,2020,Iqbal Town,Pakistan,Corolla,3700000


# Dice
It selects a sub-cube from the OLAP cube by selecting two or more dimensions. 

In [34]:
dc = revenue[(revenue['year'] ==2020) & (revenue['location'] == 'Latifabad') & 
             ((revenue['Car'] =='CIVIC') | (revenue['Car']=='Cultus')) &
            ((revenue['month']==1) | (revenue['month']==2) | (revenue['month']==3))]
dc.groupby(['year','Car','month']).sum()

,,,amount
year,Car,month,
2020,Cultus,3,2100000


# Rollup
 is about applying an aggregation function to collapse a number of dimensions. For example, we want to focus in the annual revenue for each car and collapse the location dimension (ie: we don't care where we sold our product).

It performs aggregation on the OLAP cube. 
It can be done by:
▪ Climbing up in the concept hierarchy
▪ Reducing the dimensions

In [35]:
revenue.groupby(['year']).sum().drop(['month'],axis=1)

,amount
year,
2020,1144000000
2021,1050600000
2022,1487000000


# Slice
is about fixing certain dimensions to analyze the remaining dimensions. For example, we can focus in the sales happening in "2021", "Feb", or we can focus in the sales happening in "2021", "Jan", "Car".

In [36]:
revenue[(revenue['year'] ==2021) & (revenue['month'] == 2)].head()

,month,quarter,year,location,country,Car,amount
160,2,Q1,2021,Gahli,USA,Corolla,11100000
161,2,Q1,2021,Iqbal Town,Pakistan,Corolla,11100000
162,2,Q1,2021,Gulshan,Pakistan,Cultus,6300000
163,2,Q1,2021,Gulshan,Pakistan,Swift,8400000
164,2,Q1,2021,Latifabad,Pakistan,Aqua,7500000


# Drilldown
is the reverse of "rollup" and applying an aggregation function to a finer level of granularity. For example, we want to focus in the annual and monthly revenue for each product and collapse the location dimension (ie: we don't care where we sold our product).

In drill-down operation, the less detailed data is converted into highly detailed data. 
It can be done by:
▪ Moving down in the concept hierarchy
▪ Adding a new dimension

In [37]:
revenues = revenue[revenue['Car'] == 'Aqua']
revenues.groupby(['year']).sum().drop(['month'],axis=1)

,amount
year,
2020,240000000
2021,232500000
2022,372500000


# Pivot
is about analyzing the combination of a pair of selected dimensions. For example, we want to analyze the revenue by year and month. Or we want to analyze the revenue by product and location.

It is also known as rotation operation as it rotates the current view to get a new view of the 
representation

In [38]:
revenue.pivot_table(index='Car',columns='location',values='amount')

location,AX,Gahli,Gulshan,Iqbal Town,Latifabad,Ny,Road
Car,,,,,,,
Aqua,6.250000e+06,6.351351e+06,5.555556e+06,5.810811e+06,6.145833e+06,6.071429e+06,6.111111e+06
CIVIC,1.260000e+07,1.066154e+07,8.400000e+06,1.260000e+07,1.120000e+07,1.260000e+07,1.260000e+07
Corolla,1.110000e+07,9.596875e+06,1.048333e+07,9.514286e+06,1.060667e+07,1.110000e+07,9.620000e+06
Cultus,4.200000e+06,5.311765e+06,5.600000e+06,5.415789e+06,4.846154e+06,5.775000e+06,2.100000e+06
Swift,6.872727e+06,6.855172e+06,6.676923e+06,6.565517e+06,7.466667e+06,7.866667e+06,7.400000e+06


#    --------------------------------------------------------------------------------------------------------